<a href="https://colab.research.google.com/github/anas1IA/Markov_chain/blob/main/markov_chain_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&amp;id=1zSJwAUxWv5bxyYLmYPNi-s6M_Wq5iWXh">

## Importing tools

In [61]:
import numpy as np
import pandas as pd
import os
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random

In [62]:
from google.colab import files

# Upload the homles.zip file
uploaded = files.upload()


In [63]:
from google.colab import files

# Upload the homles.zip file
uploaded = files.upload()


In [64]:
import zipfile
import os

# Specify the path to the zip file
zip_file_path = "homles.zip"

# Specify the directory where you want to extract the contents
extracted_dir_path = "my_directory"

# Create the target directory if it doesn't exist
os.makedirs(extracted_dir_path, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir_path)

# List the contents of the extracted directory
extracted_files = os.listdir(extracted_dir_path)
print("Extracted files:", extracted_files)


Extracted files: ['sherlock']


In [65]:
!mkdir my_directory

mkdir: cannot create directory ‘my_directory’: File exists


## Reading every Sherlock Holmes adventure!

In [66]:
story_path = "my_directory/sherlock/sherlock/"

def read_all_stories(story_path):
    txt = []
    for _, _, files in os.walk(story_path):
        for file in files:
            with open(story_path+file) as f:
                for line in f:
                    line = line.strip()
                    if line=='----------': break
                    if line!='':txt.append(line)
    return txt

stories = read_all_stories(story_path)
print("number of lines = ", len(stories))

number of lines =  215021


In [67]:
import nltk

# Download the 'punkt' resource
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Cleaning the text

In [68]:
def clean_txt(txt):
    cleaned_txt = []
    for line in txt:
        line = line.lower()
        line = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-\\]", "", line)
        tokens = word_tokenize(line)
        words = [word for word in tokens if word.isalpha()]
        cleaned_txt+=words
    return cleaned_txt

cleaned_stories = clean_txt(stories)
print("number of words = ", len(cleaned_stories))

number of words =  2332247


## Creating the Markov Model

In [69]:
def make_markov_model(cleaned_stories, n_gram=2):
    markov_model = {}
    for i in range(len(cleaned_stories)-n_gram-1):
        curr_state, next_state = "", ""
        for j in range(n_gram):
            curr_state += cleaned_stories[i+j] + " "
            next_state += cleaned_stories[i+j+n_gram] + " "
        curr_state = curr_state[:-1]
        next_state = next_state[:-1]
        if curr_state not in markov_model:
            markov_model[curr_state] = {}
            markov_model[curr_state][next_state] = 1
        else:
            if next_state in markov_model[curr_state]:
                markov_model[curr_state][next_state] += 1
            else:
                markov_model[curr_state][next_state] = 1

    # calculating transition probabilities
    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count/total

    return markov_model

In [70]:
markov_model = make_markov_model(cleaned_stories)

In [71]:
print("number of states = ", len(markov_model.keys()))

number of states =  208715


In [79]:
print("All possible transitions from 'my own' state: \n")
print(markov_model['my own'])

All possible transitions from 'my own' state: 

{'pursuit a': 0.005063291139240506, 'mind for': 0.005063291139240506, 'part i': 0.010126582278481013, 'way is': 0.0037974683544303796, 'hearth smoking': 0.005063291139240506, 'mind i': 0.005063291139240506, 'accord but': 0.005063291139240506, 'experience it': 0.0037974683544303796, 'professional eye': 0.0037974683544303796, 'room and': 0.0037974683544303796, 'perhaps i': 0.0037974683544303796, 'performances a': 0.0037974683544303796, 'affairs needed': 0.0037974683544303796, 'devices there': 0.0037974683544303796, 'game and': 0.007594936708860759, 'bedroom he': 0.0037974683544303796, 'against him': 0.0037974683544303796, 'story it': 0.0037974683544303796, 'story i': 0.0037974683544303796, 'face is': 0.005063291139240506, 'sake but': 0.008860759493670886, 'insignificant personality': 0.0037974683544303796, 'what were': 0.0037974683544303796, 'idea of': 0.0037974683544303796, 'way and': 0.0037974683544303796, 'time complete': 0.0037974683544

## Generating Sherlock Holmes stories!

In [73]:
def generate_story(markov_model, limit=100, start='my god'):
    n = 0
    curr_state = start
    next_state = None
    story = ""
    story+=curr_state+" "
    while n<limit:
        next_state = random.choices(list(markov_model[curr_state].keys()),
                                    list(markov_model[curr_state].values()))

        curr_state = next_state[0]
        story+=curr_state+" "
        n+=1
    return story

In [74]:
for i in range(20):
    print(str(i)+". ", generate_story(markov_model, start="dear holmes", limit=8))

0.  dear holmes oh yes with the strange business which would have been aware of a pair you should 
1.  dear holmes am i he gazed about him when i came to see him then she threw open 
2.  dear holmes what do you make of that which he himself possesses the fame of your game you 
3.  dear holmes oh yes sir i shall be most happy to look for help as we moved across 
4.  dear holmes he has acted from motives which are not a married woman grabs at her back she 
5.  dear holmes said i as i remarked before that holmes i had taken over with the roof and 
6.  dear holmes i thought there might be some very different level to your majesty there is a by 
7.  dear holmes oh yes no doubt but he attacks no one else suppose i try my luck upon 
8.  dear holmes if i had put his money if this were the plans in that safe von bork 
9.  dear holmes and tell me that it was that gaunt wasted face staring at it in my own 
10.  dear holmes said i my dear fellow we called baldy simpson and these gypsies i have als

In [75]:
for i in range(20):
    print(str(i)+". ", generate_story(markov_model, start="my dear", limit=8))

0.  my dear friend stood i know so why when we got there but you look weary yes the 
1.  my dear wife died i felt strongly that neither the country nor the sea air sunshine and patience 
2.  my dear fellow you will need your handcuffs inspector you are at your service there have come to 
3.  my dear watson said holmes with an air of displeasure her brother are the aborigines of the andaman 
4.  my dear watson said he coolly when a few weeks before my own marriage during the night a 
5.  my dear watson but the colonels face it had set himself to uncongenial silence staring moodily out of 
6.  my dear holmes am i right barker beat his head which had evidently been in his right hand 
7.  my dear watson should realize said barker quickly as he was sent a convict to siberia where now 
8.  my dear mr mac said he drop it because he foresaw that she would have forgiven me she 
9.  my dear fellow he hardly knew what he was going to say to you and that the stains 
10.  my dear daughter alice now

In [76]:
for i in range(20):
    print(str(i)+". ", generate_story(markov_model, start="i would", limit=8))

0.  i would have gone on after receiving such an injury so lightly that it was in march a 
1.  i would have swung for him and his identity had been a convict these few indications but they 
2.  i would i could discern no others sherlock holmes struck a match and in the tails of the 
3.  i would always be associated in my mind that i could find where he went to bed my 
4.  i would do what you think such a very natural hum think of her crime for which he 
5.  i would but that nothing had been done to dispose of those damned touts cried the older man 
6.  i would go said she trying hard as it seemed that the good zeppelin promises us comes true 
7.  i would have you not dad she asked so roguish and exquisite did she speak to him about 
8.  i would suggest for example that some murder has been trampled up a good many cases but i 
9.  i would ask you where is that friend hopkins will live up to his brother which he handed 
10.  i would have shown that his brother officer was in our room its

In [77]:
print(generate_story(markov_model, start="the case", limit=100))

the case is as much a departure from truth as i am ready said he there are several points in it which showed an eccentricity and irregularity in his life recently had not been drawn so that i had a chance the moon broke through the paper that lay before me but by gar if i fail i have other things to attend to will you not show you the different steps in my mind to leave a permanent weakness you know of it holmes was always of use and the rest was inconceivable you can to the morality or decency of your conduct has been here i will turn the facts which formed one of your boots although used are by no means god bless you for some weeks must elapse before we start i lay back in his purse but your other boot no sir it didnt matter to me most desperately at last he professed himself satisfied and with a very fashionably dressed and stayed for half wages it was obvious that the most powerful and richest organizations in this state that will do said mcmurdo then why did you not to do miss mor